In [1]:
from tensorflow.keras import *
import matplotlib.pyplot as plt
import numpy as np
import os
import pandas as pd
import tensorflow as tf

In [2]:
data_path = 'J:/VeReMi-Dataset/000_Select_data'
os.chdir(data_path)

In [3]:
testX_regularized=np.load('testX_processed.npy')
testY=np.load('testY_processed.npy')
trainX_regularized=np.load('trainX_processed.npy')
trainY=np.load('trainY_processed.npy')
valX_regularized=np.load('valX_processed.npy')
valY=np.load('valY_processed.npy')

In [10]:
def get_info(bucket_list, data_array):
    info_list=['type','rcvTime','pos','pos_noise','spd','spd_noise','acl','acl_noise','hed','hed_noise','sendTime','sender','senderPseudo']
    pos_list=[0,1,[2,3,4],[5,6,7],[8,9,10],[11,12,13],[14,15,16],[17,18,19],[20,21,22],[23,24,25],26,27,28,29]
    bucket_pos=[]
    for loc_buc in range(len(bucket_list)):
        for loc_info in range(len(info_list)):
            if bucket_list[loc_buc]==info_list[loc_info]:
                if isinstance(pos_list[loc_info],int):
                    bucket_pos.append(pos_list[loc_info])
                else:
                    for t in pos_list[loc_info]:
                        bucket_pos.append(t)
    # print(bucket_pos)
    result = np.empty([data_array.shape[0],len(bucket_pos)], dtype = np.float64)
    col_cnt=0
    for i in bucket_pos:
        row_cnt=0
        for j in data_array:
            result[row_cnt][col_cnt]=j[i]
            row_cnt+=1
        col_cnt+=1
    return result
# t=get_info(['type','rcvTime','pos'],valX_regularized)

[0, 1, 2, 3, 4]


In [9]:
model = Sequential()
model.add(layers.Dense(30,activation='relu',input_shape=[30]))

model.add(layers.Dropout(0.8))
model.add(layers.Dense(16,activation='relu'))
model.add(layers.Dropout(0.8))
model.add(layers.Dense(2, activation='sigmoid'))
model.compile(optimizer=optimizers.Adam(),
              loss=losses.binary_crossentropy,
              metrics=['accuracy',metrics.Precision(),metrics.Recall(),metrics.F1])

NameError: name 'recall_score' is not defined

In [11]:
model.summary()

Model: "sequential_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_3 (Dense)              (None, 30)                930       
_________________________________________________________________
dropout_2 (Dropout)          (None, 30)                0         
_________________________________________________________________
dense_4 (Dense)              (None, 16)                496       
_________________________________________________________________
dropout_3 (Dropout)          (None, 16)                0         
_________________________________________________________________
dense_5 (Dense)              (None, 2)                 34        
Total params: 1,460
Trainable params: 1,460
Non-trainable params: 0
_________________________________________________________________


In [7]:
trainY=tf.one_hot(trainY,2)
valY=tf.one_hot(valY,2)
testY_m=tf.one_hot(testY,2)

In [10]:
history = model.fit(trainX_regularized, trainY, batch_size=64, epochs=3, validation_data=(valX_regularized, valY))

RuntimeError: You must compile your model before training/testing. Use `model.compile(optimizer, loss)`.

In [ ]:
effect = model.evaluate(testX_regularized, testY_m)

In [ ]:
#testX_regularized
y_pred=model.predict(testX_regularized,batch_size=1)
y_pred_label=[]
for i in y_pred:
    if i[0]>i[1]:
        y_pred_label.append(0.0)
    else:
        y_pred_label.append(13.0)

In [ ]:
right=0
for loc in range (223972):
    if testY[loc]==y_pred_label[loc]*1.0:
        right+=1
acc=right/223972
print(acc)